In [112]:
!pip install pandas

In [113]:
!.\tf_env\Scripts\python.exe -m pip install "pandas"

In [114]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

In [115]:
!.\tf_env\Scripts\python.exe -m pip install "scikit-learn"

In [116]:
df = pd.read_csv("C:/Users/devik/Downloads/insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [117]:
X = df.drop("bought_insurance",axis="columns")

In [118]:
X.head(5)

,age,affordibility
0,22,1
1,25,0
2,47,1
3,52,0
4,46,1


In [119]:
Y=df.bought_insurance

In [120]:
Y.head(5)

0    0
1    0
2    1
3    0
4    1
Name: bought_insurance, dtype: int64

In [121]:
import sklearn
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,train_size=0.2,random_state=23)

In [163]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [123]:
X_test_scaled.head(5)

,age,affordibility
18,0.19,0
4,0.46,1
23,0.45,1
24,0.50,1
3,0.52,0


In [164]:
X_train_scaled.head(5)

,age,affordibility
22,0.40,1
9,0.61,1
8,0.62,1
6,0.55,0
19,0.18,1


In [ ]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, Y_train, epochs=5000)

Epoch 1/5000
1/1 [==============================] - 1s 847ms/step - loss: 0.6073 - accuracy: 0.6000
Epoch 2/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.6071 - accuracy: 0.6000
Epoch 3/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.6068 - accuracy: 0.6000
Epoch 4/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.6065 - accuracy: 0.6000
Epoch 5/5000
1/1 [==============================] - 0s 5ms/step - loss: 0.6063 - accuracy: 0.6000
Epoch 6/5000
1/1 [==============================] - 0s 10ms/step - loss: 0.6060 - accuracy: 0.6000
Epoch 7/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.6058 - accuracy: 0.6000
Epoch 8/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.6055 - accuracy: 0.6000
Epoch 9/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.6053 - accuracy: 0.6000
Epoch 10/5000
1/1 [==============================] - 0s 7ms/step - loss: 0.6050 - accuracy: 0.6000
Epoch 11/5000
1/

In [165]:
model.evaluate(X_test_scaled,Y_test)

1/1 [==============================] - 0s 42ms/step - loss: 0.5225 - accuracy: 0.6957


[0.5224924683570862, 0.695652186870575]

In [166]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 41ms/step


array([[0.03916065],
       [0.78827   ],
       [0.7799037 ],
       [0.81938994],
       [0.17233318],
       [0.78827   ],
       [0.8119601 ],
       [0.8531262 ],
       [0.48267916],
       [0.87074953],
       [0.5320511 ],
       [0.05197601],
       [0.85921156],
       [0.6046593 ],
       [0.79640126],
       [0.05197601],
       [0.51972914],
       [0.05446591],
       [0.544334  ],
       [0.23616919],
       [0.05706793],
       [0.06262574],
       [0.84682477]], dtype=float32)

In [144]:
coef, intercept = model.get_weights()
coef, intercept

(array([[4.9422846],
        [3.1802464]], dtype=float32),
 array([-4.1391687], dtype=float32))

In [145]:
def sigmoid(x):
        import math
        return 1 / (1 + math.exp(-x))
sigmoid(18)

0.9999999847700205

In [146]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

prediction_function(.47, 1)

0.7964011991607933

In [147]:
prediction_function(.18, 1)

0.48267914811694573

In [148]:
def sigmoid_numpy(X):
   return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

In [150]:
def log_loss(y_true, y_predicted):
    epsilon = 1e-15
    y_predicted_new = [max(i,epsilon) for i in y_predicted]
    y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new = np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [151]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = sigmoid_numpy(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [152]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],Y_train,1000, 0.4631)

Epoch:0, w1:0.9796707338642361, w2:0.9764661791443727, bias:-0.08694737995670729, loss:0.6073308808934209
Epoch:1, w1:0.9639496965304336, w2:0.9605408122701444, bias:-0.1639811581531168, loss:0.591399915721739
Epoch:2, w1:0.9523429209080886, w2:0.9513955228516378, bias:-0.2321802788130915, loss:0.5793763468437586
Epoch:3, w1:0.9443277587898457, w2:0.9481328103880075, bias:-0.29268197452725175, loss:0.5702540302793787
Epoch:4, w1:0.9393917777182781, w2:0.9498602118422911, bias:-0.3465978826008559, loss:0.5632207176481923
Epoch:5, w1:0.9370587481987119, w2:0.9557410146625793, bias:-0.3949585515809945, loss:0.55765630657171
Epoch:6, w1:0.9369029435691465, w2:0.9650231875878842, bias:-0.4386833396491154, loss:0.553107485972731
Epoch:7, w1:0.9385545265591344, w2:0.9770514227707401, bias:-0.478569500146825, loss:0.5492537830680205
Epoch:8, w1:0.9416989815035017, w2:0.9912677242225126, bias:-0.515293974888884, loss:0.5458743607360304
Epoch:9, w1:0.9460730601923354, w2:1.0072051572518457, bias

(1.3914019670308029, 1.7830839113613464, -1.4874400139194146)

In [153]:
coef, intercept

(array([[4.9422846],
        [3.1802464]], dtype=float32),
 array([-4.1391687], dtype=float32))

In [155]:
class myNN:
    def __init__(self):
        self.w1 = 1 
        self.w2 = 1
        self.bias = 0
        
    def fit(self, X, y, epochs, loss_thresold):
        self.w1, self.w2, self.bias = self.gradient_descent(X['age'],X['affordibility'],y, epochs, loss_thresold)
        print(f"Final weights and bias: w1: {self.w1}, w2: {self.w2}, bias: {self.bias}")
        
    def predict(self, X_test):
        weighted_sum = self.w1*X_test['age'] + self.w2*X_test['affordibility'] + self.bias
        return sigmoid_numpy(weighted_sum)

    def gradient_descent(self, age,affordability, y_true, epochs, loss_thresold):
        w1 = w2 = 1
        bias = 0
        rate = 0.5
        n = len(age)
        for i in range(epochs):
            weighted_sum = w1 * age + w2 * affordability + bias
            y_predicted = sigmoid_numpy(weighted_sum)
            loss = log_loss(y_true, y_predicted)
            
            w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
            w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

            bias_d = np.mean(y_predicted-y_true)
            w1 = w1 - rate * w1d
            w2 = w2 - rate * w2d
            bias = bias - rate * bias_d
            
            if i%50==0:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
            
            if loss<=loss_thresold:
                print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')
                break

        return w1, w2, bias

In [156]:
customModel = myNN()
customModel.fit(X_train_scaled, Y_train, epochs=8000, loss_thresold=0.4631)

Epoch:0, w1:0.9796707338642361, w2:0.9764661791443727, bias:-0.08694737995670729, loss:0.6073308808934209
Epoch:50, w1:1.3672718893264535, w2:1.7533892195481533, bias:-1.4509853077592925, loss:0.4649261559794001
Epoch:52, w1:1.3914019670308029, w2:1.7830839113613464, bias:-1.4874400139194146, loss:0.4621109135202331
Final weights and bias: w1: 1.3914019670308029, w2: 1.7830839113613464, bias: -1.4874400139194146


In [157]:
coef, intercept

(array([[4.9422846],
        [3.1802464]], dtype=float32),
 array([-4.1391687], dtype=float32))

In [158]:
X_test_scaled

,age,affordibility
18,0.19,0
4,0.46,1
23,0.45,1
24,0.50,1
3,0.52,0
27,0.46,1
14,0.49,1
15,0.55,1
10,0.18,1
17,0.58,1


In [159]:
customModel.predict(X_test_scaled)

18    0.227396
4     0.718228
23    0.715404
24    0.729353
3     0.317798
27    0.718228
14    0.726598
15    0.742865
10    0.633229
17    0.750758
0     0.646057
16    0.242396
5     0.745514
11    0.664906
2     0.721035
1     0.242396
20    0.642869
21    0.244960
26    0.649232
7     0.342404
12    0.247543
13    0.252762
25    0.740199
dtype: float64

In [160]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 35ms/step


array([[0.03916065],
       [0.78827   ],
       [0.7799037 ],
       [0.81938994],
       [0.17233318],
       [0.78827   ],
       [0.8119601 ],
       [0.8531262 ],
       [0.48267916],
       [0.87074953],
       [0.5320511 ],
       [0.05197601],
       [0.85921156],
       [0.6046593 ],
       [0.79640126],
       [0.05197601],
       [0.51972914],
       [0.05446591],
       [0.544334  ],
       [0.23616919],
       [0.05706793],
       [0.06262574],
       [0.84682477]], dtype=float32)